In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [2]:
%cd ..

/home/jan/misc/copenhagen-hack/optimal-ph/aux/model-playground


In [3]:
train_meta = pd.read_csv("../train.csv", index_col=0)
valid_meta = pd.read_csv("../valid.csv", index_col=0)
train_meta

,mean_growth_PH,sequence,representative,is7
99973,7.40,MEANHGMNNYIKLAFVFGITTMATSYADTVAPPTLLTAQKLPQLQQ...,True,False
84793,7.00,MEFFKKTALAALVMGFSGAALALPNITILATGGTIAGGGDSATKSN...,False,True
27864,6.50,MSPLGILRRHRVAALLGAALIISPVVVSFAQSANSTGVSKIVATTQ...,True,False
46228,7.80,MKKQYWYVIITYVAMQLSSLVGVPLLAHSGFINASNKDIAISIASG...,False,False
6028,7.00,MSKKKMAITLSAMLSATIIPSFTMDVHAEKKEETKNTKIELENGMT...,False,True
...,...,...,...,...
69002,7.00,MEIIMRNLCFLLTLVATLLLHGRLIAAALPQDEKLITGQLDNGLRY...,False,True
75674,7.00,MSKHPKLLVLALACLACAGRASAAPASDEVARLAQRCAPDVSPLTM...,False,True
50377,7.45,MSRAGSLMLVLGTALWLCGCSGMNSENKRVAPVAEKRPHTMSLHGV...,True,False
87875,7.00,MSAGRLNKKSLGIVMLLSVGLLLAGCSGSKSSDTGTYSGSVYTVKR...,False,True


In [6]:
freq1 = pd.read_csv("data/1-mers.tsv", sep="\t")
freq2 = pd.read_parquet("data/2-mers.parquet.gzip")
phychem = pd.read_csv("../../data/physchem/properties.csv", index_col=0).drop("ID", axis=1).reset_index(drop=True)
features = pd.concat([freq1, freq2, phychem], axis=1)


In [7]:
def get_xy(metadata, features, only_repr = False, only_not7 = False):
    if only_repr:
        metadata = metadata[metadata.representative]
    if only_not7:
        metadata = metadata[~metadata.is7]
    
    y = metadata.mean_growth_PH.to_numpy()
    X = features.loc[metadata.index].to_numpy()
    return X, y

X_train, y_train = get_xy(train_meta, features, only_not7=True)
X_valid, y_valid = get_xy(valid_meta, features, only_not7=True)


In [8]:
X_train.shape, y_train.shape

((28168, 425), (28168,))

In [9]:
X_valid.shape, y_valid.shape

((7042, 425), (7042,))

In [10]:
X_train_repr, y_train_repr = get_xy(train_meta, features, only_not7=True, only_repr=True)
X_valid_repr, y_valid_repr = get_xy(valid_meta, features, only_not7=True, only_repr=True)

X_train_repr = X_train_repr
X_valid_repr = X_valid_repr


X_train_repr.shape, X_valid_repr.shape

((14209, 425), (3552, 425))

In [11]:
models = [
    ("RandomForest d4", RandomForestRegressor(max_depth=4, random_state=31415)),
]

for name, model in models:
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    valid_repr_pred = model.predict(X_valid_repr)


    print(">>", name)
    print("Train | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(train_pred, y_train)[0], np.sqrt(mean_squared_error(train_pred, y_train))))
    print("Valid | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_pred, y_valid)[0], np.sqrt(mean_squared_error(valid_pred, y_valid))))
    print("ValRe | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_repr_pred, y_valid_repr)[0], np.sqrt(mean_squared_error(valid_repr_pred, y_valid_repr))))

>> RandomForest d4
Train | Spearman 0.3649 RMSE 0.9857
Valid | Spearman 0.3496 RMSE 0.9814
ValRe | Spearman 0.3545 RMSE 1.1109
